Plot the spectra of our linear operators. They have very weird spectra.

In [ ]:
from sklearn.externals import joblib
import xarray as xr
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline


import holoviews as hv
import datashader as ds
hv.extension('matplotlib')

In [ ]:
def eigval_plot(lm_data):
    mat = lm_data['mat']

    # get pushforward operator
    A = 3/24*mat + np.eye(mat.shape[0])

    lam, P = np.linalg.eig(A)

    return hv.Points((lam.real, lam.imag), kdims=[r'$\Re$', r'$\Im$']) * hv.Ellipse(0,0,2)

In [ ]:
eigval_plot(joblib.load("../data/ml/ngaqua/mca_regression.pkl"))

In [ ]:
%%opts Points[aspect=2]
eigval_plot(joblib.load("../data/ml/ngaqua/linear_model.pkl"))

It is definitely a problem that these modes are all growing!

In [ ]:
A = joblib.load("../data/ml/ngaqua/mca_regression.pkl")['mat']
lam, P = np.linalg.eig(A)

sort_idx = np.argsort(np.abs(lam))
P = P[:, sort_idx[::-1]]
lam = lam[sort_idx[::-1]]

In [ ]:
%%output size=50
%%opts Curve[invert_axes=True, aspect=.25] {+axiswise +framewise}

hmap = hv.HoloMap(kdims=["Mode", "Component"])

for k, y in enumerate(P.T[:10]):
    y = y/y[0]
    lam_lab = "%.2f + i%.2f"%(lam[k].real, lam[k].imag)
    hmap[k, "r"]= hv.Curve(np.abs(y), kdims="z") * hv.Text(0,0, lam_lab)
    
    tht = np.arctan(y.imag/y.real)
    hmap[k, "tht"]= hv.Overlay((hv.Curve(tht, kdims="z", label=lam_lab),))
    

hmap.layout("Component")